## Exploring Sound Generation with Python wave module

Have you ever stopped and wondered how a computer is able to play a sound file or how we are able to record and store sounds?
How does a sequence of zeroes and ones get converted into physical sound that come out through the speakers?
How are all these real-life analogue sound saved as zeroes and ones?

In this notebook, we will explore the physics of sound and how the information is stored digitally inside a raw WAV file.

1. Writing a WAV file
2. Understanding the data format
3. Physics of sound
4. Representing a note
5. Representing a melody
6. Writing a simple song and saving as a WAV

### 1. Writing a WAV file

In this notebook, we will be using a standard Python module called *wave* that provides basic file writing and reading functionalities.
You can also read a [good introduction to generating wave files](http://soledadpenades.com/2009/10/29/fastest-way-to-generate-wav-files-in-python-using-the-wave-module/) written by [Soledad Penades](https://github.com/sole)

### 2. Understanding the data format



### 3. Physics of sound

### 4. Representing a note

### 5. Representing a melody

### 6. Writing a simple song and saving as a WAV